# Airbnb New User Bookings - Gradient Boosting with H2O

Load the required libraries

In [1]:
import h2o
import pandas as pd
import numpy as np

Initialize H2O cluster:

In [2]:
h2o.init()
h2o.remove_all() 



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /tmp/tmpB9uG0f/h2o_david_started_from_python.out
JVM stderr: /tmp/tmpUbeObW/h2o_david_started_from_python.err
Using ice_root: /tmp/tmp3uQrKq


Java Version: java version "1.7.0_91"
OpenJDK Runtime Environment (IcedTea 2.6.3) (7u91-2.6.3-0ubuntu0.14.04.1)
OpenJDK 64-Bit Server VM (build 24.91-b01, mixed mode)


Starting H2O JVM and connecting: ............ Connection successful!


H2O cluster uptime:,1 seconds 258 milliseconds
H2O cluster version:,3.6.0.3
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.72 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


## Data Loading

In [3]:
train_users = h2o.import_file("../datasets/processed/preprocessed.csv")


Parse Progress: [##################################################] 100%


Remove train users's ID's since we don't need them to train our classifier

In [4]:
train_users = train_users.drop('id')

In [5]:
train_users = train_users.drop('C1')

In [31]:
train_users.nrow

275547

In [28]:
len(a.columns)

200

In [6]:
X = train_users.columns
del X[train_users.columns.index('country_destination')]

In [7]:
# X =[
#     u'date_account_created',
#     u'timestamp_first_active',
#     u'gender',
#     u'age',
#     u'signup_method',
#     u'signup_flow',
#     u'language',
#     u'affiliate_channel',
#     u'affiliate_provider',
#     u'first_affiliate_tracked',
#     u'signup_app',
#     u'first_device_type',
#     u'first_browser',
# #     u'country_destination'
# ]

y = u'country_destination'

In [8]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator

gbe = H2OGradientBoostingEstimator(
    distribution='multinomial', 
#     min_rows=None, 
#     nbins=None,
#     sample_rate=None, 
#     col_sample_rate=None, 
#     nbins_top_level=None, 
#     nbins_cats=None, 
#     balance_classes=None,
#     max_after_balance_size=None, 
#     seed=None, 
#     build_tree_one_node=None,
    nfolds=10, 
#     fold_assignment=None, 
#     keep_cross_validation_predictions=None,
#     stopping_rounds=None, 
#     stopping_metric=None, 
#     stopping_tolerance=None,
#     score_each_iteration=None, 
#     checkpoint=None
)

hyper_parameters = {
    "ntrees": [1], 
    "max_depth": [1], 
    "learn_rate": [0.1], 
}

gs = H2OGridSearch(gbe, hyper_params=hyper_parameters)

In [9]:
gs.train(X, y, training_frame=train_users)


gbm Grid Build Progress: [                                                  ] 00%

KeyboardInterrupt: 

In [ ]:
gs.r2()

### Predictions

In [ ]:
test_users = h2o.import_file("../data/raw/test_users.csv")

In [ ]:
predictions = dl.predict(test_users.drop('id'))

### Make CSV for submission

In [ ]:
# Load the predictions into a DataFrame
country_predictions = predictions.as_data_frame()
country_predictions = pd.DataFrame(country_predictions).transpose()

# Use first row as column names
country_predictions.columns = country_predictions.iloc[0]
country_predictions = country_predictions.reindex(country_predictions.index.drop(0))

# Drop labeled prediction
country_predictions.drop('predict', axis=1, inplace=True)

In [ ]:
# Load the test users into a DataFrame
test_users = pd.DataFrame(test_users.as_data_frame()).transpose()

# Use first row as column names
test_users.columns = test_users.iloc[0]
test_users = test_users.reindex(test_users.index.drop(0))

In [ ]:
submission = []
number_of_users = country_predictions.shape[0]

# Iterate over each user
for user in range(number_of_users):
    # Get the 5 most provable destination countries
    user_prediction = country_predictions.iloc[user].astype(float)
    
    # Sort in descending order
    user_prediction = user_prediction.sort_values(ascending=False)
    
    # Append the 5 with higher provability
    for country in range(5):
        user_id = test_users.iloc[user]['id']
        destination = list(user_prediction.index[0:5])[country]
        submission.append([user_id, destination])

In [ ]:
sub_file = pd.DataFrame(submission, columns = ['id', 'country'])

Write CSV file:

In [ ]:
sub_file.to_csv('../data/submissions/h2o-gb.csv', index=False)

In [ ]:
h2o.shutdown()